## Importações

In [2]:
import pandas as pd
import os
import ast
import locale
from datetime import datetime
from dateutil.relativedelta import relativedelta

from constants import (
    DIR_GAMES_DATA, DIR_MOVIES_SHOW_DATA, DIR_OUTPUT_TABLES, MOVIES_FILE_NAME, SHOWS_FILE_NAME
)

## Definição de variáveis gerais

In [3]:
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')


'en_US.UTF-8'

## Carregando arquivos csv

### Arquivos de jogos
Carregar csv em dataframes, que são associados a dicionários

In [4]:
data_dict: dict[str, dict[str, pd.DataFrame]] = dict()
cont = 0
for root , subdirs, files in os.walk(DIR_GAMES_DATA):
    for file in files:
        file_path = os.path.join(root, file)
        franchise_name = os.path.normpath(root).split(os.sep)[-1]
        if data_dict.get(franchise_name) is None:
            data_dict[franchise_name] = dict()
        if file.endswith('.csv'):
            df = pd.read_csv(
                file_path,
                converters={
                    'Peak': lambda x: int(x.replace(',', '').replace('+', ''))
                            if x.replace(',', '').replace('+', '').lstrip('-').isdigit() 
                            else None,
                    'Gain': lambda x: int(x.replace(',', '').replace('+', ''))
                            if x.replace(',', '').replace('+', '').lstrip('-').isdigit() 
                            else None,
                    '% Gain': lambda x: float(x.replace('+', '').replace('%', ''))
                              if x.replace('+', '').replace('%', '').lstrip('-').replace('.', '', 1).isdigit()
                              else None,
                    'Average': lambda x: int(x.replace(',', '').replace('+', ''))
                               if x.replace(',', '').replace('+', '').lstrip('-').isdigit() 
                               else None,
                    'Avg % Gain': lambda x: float(x.replace('+', '').replace('%', ''))
                                  if x.replace('+', '').replace('%', '').lstrip('-').replace('.', '', 1).isdigit()
                                  else None,
                    
                }
            )
            # Converte coluna 'Month' com valores em string para datetime.date
            df['Month'] = pd.to_datetime(df['Month'], format='%B %Y', errors='coerce').dt.date
            # remove linhas onde Month é NaT
            df = df.dropna(subset=['Month'])
            # print(f"Carregando {file} de {file_path} com {len(df)} linhas.")
            data_dict[franchise_name][file] = df
            cont += 1
print(f"Total de arquivos carregados: {cont}")

Total de arquivos carregados: 79


### Arquivos de filmes/séries
Cada filme/série dos arquivos csv são atribuídos a chave correspondente da franquia no dicionário

In [ ]:
df_movies = pd.read_csv(
    os.path.join(DIR_MOVIES_SHOW_DATA, MOVIES_FILE_NAME),
)
# Converte coluna 'ReleaseDate' com valores em string para datetime.date
df_movies['ReleaseDate'] = pd.to_datetime(df_movies['ReleaseDate'], format='%B %Y', errors='coerce').dt.date

df_shows = pd.read_csv(
    os.path.join(DIR_MOVIES_SHOW_DATA, SHOWS_FILE_NAME),
    converters={
        'SeasonsReleaseDate': ast.literal_eval,
        'RatingSeasons': ast.literal_eval,
        'VotesSeasons': ast.literal_eval,
        # adicione aqui outras colunas em que queira fazer o mesmo
    }
)
# Converte coluna 'SeasonsReleaseDate' com valores em lista de strings para lista de datetime.date
df_shows['SeasonsReleaseDate'] = df_shows['SeasonsReleaseDate'].apply(
    lambda lst: [
        pd.to_datetime(s, format='%B %Y', errors='coerce').date()
        for s in lst
    ]
)

for idx, row in df_movies.iterrows():
    data_dict[row['Franchise']][MOVIES_FILE_NAME] = row

for idx, row in df_shows.iterrows():
    data_dict[row['Franchise']][SHOWS_FILE_NAME] = row

print("Adaptações carregadas com sucesso.")

Adaptações carregadas com sucesso.


## Extraindo crescimento dos jogos
Geraremos um dataframe que relaciona jogos e adaptações cinematográficas de franquias famosas com as colunas:
- **GameTitle**: Título do jogo
- **AdaptTitle**: Título da adaptação cinematográfica
- **Type**: tipo da adaptação (movie ou show)
- **ReleaseDate**: Data em que a adaptação cinematográfica foi lançada
- **Franchise**: Franquia a qual o jogo e a adaptação pertencem 
- **AdaptReleaseAbsoluteGrowth**: Crescimento da base de jogadores (valores absolutos) no mês de lançamento da adaptação cinematográfica
- **AdaptReleasePercentualGrowth**: Crescimento da base de jogadores (percentual, relativo ao mês anterior) no mês de lançamento da adaptação cinematográfica
- **PreviousYearAbsoluteGrowth**: Crescimento da base de jogadores (valores absolutos) um ano antes no mesmo mês
- **PreviousYearPercentualGrowth**: Crescimento da base de jogadores (percentual, relativo ao mês anterior) um ano antes no mesmo mês
- **NextMonthAbsoluteGrowth**: Crescimento da base de jogadores (valores absolutos) no mês seguinte ao lançamento da adaptação cinematográfica
- **NextMonthPercentualGrowth**: Crescimento da base de jogadores (percentual, relativo ao mês anterior) no mês seguinte ao lançamento da adaptação cinematográfica
- **NextMonthPreviousYearAbsoluteGrowth**: Crescimento da base de jogadores (valores absolutos) no ano anterior, mês seguinte ao lançamento da adaptação cinematográfica
- **NextMonthPreviousYearPercentualGrowth**:  Crescimento da base de jogadores (percentual, relativo ao mês anterior) no ano anterior, mês seguinte ao lançamento da adaptação cinematográfica
- **Rating**: Nota/avaliação dos espectadores da adaptação cinematográfica (de 0 a 10)
- **NumVotes(k)**: Número de avaliações (em milhares) da adaptação cinematográfica

In [ ]:
df_game_adaptation = pd.DataFrame(
    columns=[
        'GameTitle', 'MaxHistoricalPeak',
        'AdaptReleasePeak', 'AdaptReleaseAbsoluteGrowth', 'AdaptReleasePercentualGrowth',
        'PreviousYearPeak', 'PreviousYearAbsoluteGrowth', 'PreviousYearPercentualGrowth',
        'NextMonthPeak', 'NextMonthAbsoluteGrowth', 'NextMonthPercentualGrowth', 
        'NextMonthPreviousYearPeak', 'NextMonthPreviousYearAbsoluteGrowth', 'NextMonthPreviousYearPercentualGrowth',
        'Franchise', 'Type', 'AdaptTitle', 'ReleaseDate', 'Rating', 'NumVotes(k)',
    ]
)

for franchise, franchise_files in data_dict.items():
    
    # Caso 1: Adaptações como filmes
    if MOVIES_FILE_NAME in franchise_files:
        movie_df = franchise_files[MOVIES_FILE_NAME]
        
        for game_file_name, game_df in franchise_files.items():
            if game_file_name == MOVIES_FILE_NAME:  # Se for o arquivo de filmes, pula
                continue
            
            # GameTitle
            gameTitle = game_file_name.removesuffix('_chart_month_data.csv').replace('_', ' ')
            
            # Type
            adapt_type = 'Movie'
            
            #AdaptTitle
            adaptTitle = movie_df['Title']
            
            # ReleaseDate
            releaseDate = movie_df['ReleaseDate']
            
            # Rating
            rating = movie_df['Rating']
            
            # NumVotes(k)
            numVotes = movie_df['Votes']
            
            # MaxHistoricalPeak
            game_df_before_release = game_df[ game_df['Month'] < releaseDate ]
            maxHistoricalPeak = game_df_before_release['Peak'].max() if not game_df_before_release.empty else None
            
            # AdaptReleasePeak, AdaptReleaseAbsoluteGrowth e AdaptReleasePercentualGrowth
            game_df_releaseDate = game_df[ game_df['Month'] == releaseDate ]
            if not game_df_releaseDate.empty:
                adaptReleasePeak = game_df_releaseDate['Peak'].iloc[0]
                adaptReleaseAbsoluteGrowth = game_df_releaseDate['Gain'].iloc[0]
                adaptReleasePercentualGrowth = game_df_releaseDate['% Gain'].iloc[0]
            else:
                adaptReleasePeak = None
                adaptReleaseAbsoluteGrowth = None
                adaptReleasePercentualGrowth = None
                
            # NextMonthPeak, NextMonthAbsoluteGrowth e NextMonthPercentualGrowth
            game_df_next_month = game_df[ game_df['Month'] == (releaseDate + relativedelta(months=1)) ]
            if not game_df_next_month.empty:
                nextMonthPeak = game_df_next_month['Peak'].iloc[0]
                nextMonthAbsoluteGrowth = game_df_next_month['Gain'].iloc[0]
                nextMothPercentualGrowth = game_df_next_month['% Gain'].iloc[0]
            else:
                nextMonthPeak = None
                nextMonthAbsoluteGrowth = None
                nextMothPercentualGrowth = None
            
            # PreviousYearPeak, PreviousYearAbsoluteGrowth e PreviousYearPercentualGrowth
            game_df_previous_year = game_df[ game_df['Month'] == (releaseDate - relativedelta(years=1)) ]
            if not game_df_previous_year.empty:
                previousYearPeak = game_df_previous_year['Peak'].iloc[0]
                previousYearAbsoluteGrowth = game_df_previous_year['Gain'].iloc[0]
                previousYearPercentualGrowth = game_df_previous_year['% Gain'].iloc[0]
            else:
                previousYearPeak = None
                previousYearAbsoluteGrowth = None
                previousYearPercentualGrowth = None
                
            # NextMonthPreviousYearPeak, NextMonthPreviousYearAbsoluteGrowth e NextMonthPreviousYearPercentualGrowth
            game_df_next_month_previous_year = game_df[ game_df['Month'] == (releaseDate - relativedelta(years=1) + relativedelta(months=1)) ]
            if not game_df_next_month_previous_year.empty:
                nextMonthPreviousYearPeak = game_df_next_month_previous_year['Peak'].iloc[0]
                nextMonthPreviousYearAbsoluteGrowth = game_df_next_month_previous_year['Gain'].iloc[0]
                nextMonthPreviousYearPercentualGrowth = game_df_next_month_previous_year['% Gain'].iloc[0]
            else:
                nextMonthPreviousYearPeak = None
                nextMonthPreviousYearAbsoluteGrowth = None
                nextMonthPreviousYearPercentualGrowth = None
                
            # Adiciona a linha ao DataFrame df_game_adaptation 
            new_row = pd.DataFrame([{
                'GameTitle': gameTitle,
                'MaxHistoricalPeak': maxHistoricalPeak,
                'AdaptReleasePeak': adaptReleasePeak,
                'AdaptReleaseAbsoluteGrowth': adaptReleaseAbsoluteGrowth,
                'AdaptReleasePercentualGrowth': adaptReleasePercentualGrowth,
                'PreviousYearPeak': previousYearPeak,
                'PreviousYearAbsoluteGrowth': previousYearAbsoluteGrowth,
                'PreviousYearPercentualGrowth': previousYearPercentualGrowth,
                'NextMonthPeak': nextMonthPeak,
                'NextMonthAbsoluteGrowth': nextMonthAbsoluteGrowth,
                'NextMonthPercentualGrowth': nextMothPercentualGrowth,
                'NextMonthPreviousYearAbsoluteGrowth': nextMonthPreviousYearAbsoluteGrowth,
                'NextMonthPreviousYearPercentualGrowth': nextMonthPreviousYearPercentualGrowth,
                'NextMonthPreviousYearPeak': nextMonthPreviousYearPeak,
                'Franchise': franchise,
                'Type': adapt_type,
                'AdaptTitle': adaptTitle,
                'ReleaseDate': releaseDate,
                'Rating': rating,
                'NumVotes(k)': numVotes
            }])
            df_game_adaptation = pd.concat([df_game_adaptation, new_row], ignore_index=True)

    # Caso 2: Adaptações como séries
    if SHOWS_FILE_NAME in franchise_files:
        show_df = franchise_files[SHOWS_FILE_NAME]
        
        # Para cada jogo na franquia (arquivos csv dos jogos)
        for game_file_name, game_df in franchise_files.items():
            if game_file_name == SHOWS_FILE_NAME:  # Se for o arquivo de séries, pula
                continue
            
            # GameTitle
            gameTitle = game_file_name.removesuffix('_chart_month_data.csv').replace('_', ' ')
            
            # Type
            adapt_type = 'Show'
            
            seasonsReleaseDate: list[datetime] = show_df['SeasonsReleaseDate']
            
            # Para cada temporada da série (adaptação do jogo)
            for i, season_date in enumerate(seasonsReleaseDate):
            
                # AdaptTitle
                adaptTitle = show_df['Title'] + f" - S{i+1:02d}"
                
                # ReleaseDate
                releaseDate = season_date
                
                # Rating
                rating = show_df['RatingSeasons'][i] if i < len(show_df['RatingSeasons']) else None
                
                # NumVotes(k)
                numVotes = show_df['VotesSeasons'][i] if i < len(show_df['VotesSeasons']) else None
                
                # MaxHistoricalPeak
                game_df_before_release = game_df[ game_df['Month'] < releaseDate ]
                maxHistoricalPeak = game_df_before_release['Peak'].max() if not game_df_before_release.empty else None
                
                # AdaptReleasePeak, AdaptReleaseAbsoluteGrowth e AdaptReleasePercentualGrowth
                game_df_releaseDate = game_df[ game_df['Month'] == season_date]
                if not game_df_releaseDate.empty:
                    adaptReleasePeak = game_df_releaseDate['Peak'].iloc[0]
                    adaptReleaseAbsoluteGrowth = game_df_releaseDate['Gain'].iloc[0]
                    adaptReleasePercentualGrowth = game_df_releaseDate['% Gain'].iloc[0]
                else:
                    adaptReleasePeak = None
                    adaptReleaseAbsoluteGrowth = None
                    adaptReleasePercentualGrowth = None
                
                # NextMonthPeak, NextMonthAbsoluteGrowth e NextMonthPercentualGrowth
                game_df_next_month = game_df[ game_df['Month'] == (season_date + relativedelta(months=1)) ]
                if not game_df_next_month.empty:
                    nextMonthPeak = game_df_next_month['Peak'].iloc[0]
                    nextMonthAbsoluteGrowth = game_df_next_month['Gain'].iloc[0]
                    nextMothPercentualGrowth = game_df_next_month['% Gain'].iloc[0]
                else:
                    nextMonthPeak = None
                    nextMonthAbsoluteGrowth = None
                    nextMothPercentualGrowth = None
                    
                # PreviousYearPeak, PreviousYearAbsoluteGrowth e PreviousYearPercentualGrowth
                game_df_previous_year = game_df[ game_df['Month'] == (season_date - relativedelta(years=1)) ]
                if not game_df_previous_year.empty:
                    previousYearPeak = game_df_previous_year['Peak'].iloc[0]
                    previousYearAbsoluteGrowth = game_df_previous_year['Gain'].iloc[0]
                    previousYearPercentualGrowth = game_df_previous_year['% Gain'].iloc[0]
                else:
                    previousYearPeak = None
                    previousYearAbsoluteGrowth = None
                    previousYearPercentualGrowth = None
                
                # NextMonthPreviousYearPeak, NextMonthPreviousYearAbsoluteGrowth e NextMonthPreviousYearPercentualGrowth
                game_df_next_month_previous_year = game_df[ game_df['Month'] == (season_date - relativedelta(years=1) + relativedelta(months=1)) ]
                if not game_df_next_month_previous_year.empty:
                    nextMonthPreviousYearPeak = game_df_next_month_previous_year['Peak'].iloc[0]
                    nextMonthPreviousYearAbsoluteGrowth = game_df_next_month_previous_year['Gain'].iloc[0]
                    nextMonthPreviousYearPercentualGrowth = game_df_next_month_previous_year['% Gain'].iloc[0]
                else:
                    nextMonthPreviousYearPeak = None
                    nextMonthPreviousYearAbsoluteGrowth = None
                    nextMonthPreviousYearPercentualGrowth = None
                    
                # Adiciona a linha ao DataFrame df_game_adaptation 
                new_row = pd.DataFrame([{
                    'GameTitle': gameTitle,
                    'MaxHistoricalPeak': maxHistoricalPeak,
                    'AdaptReleasePeak': adaptReleasePeak,
                    'AdaptReleaseAbsoluteGrowth': adaptReleaseAbsoluteGrowth,
                    'AdaptReleasePercentualGrowth': adaptReleasePercentualGrowth,
                    'PreviousYearPeak': previousYearPeak,
                    'PreviousYearAbsoluteGrowth': previousYearAbsoluteGrowth,
                    'PreviousYearPercentualGrowth': previousYearPercentualGrowth,
                    'NextMonthPeak': nextMonthPeak,
                    'NextMonthAbsoluteGrowth': nextMonthAbsoluteGrowth,
                    'NextMonthPercentualGrowth': nextMothPercentualGrowth,
                    'NextMonthPreviousYearPeak': nextMonthPreviousYearPeak,
                    'NextMonthPreviousYearAbsoluteGrowth': nextMonthPreviousYearAbsoluteGrowth,
                    'NextMonthPreviousYearPercentualGrowth': nextMonthPreviousYearPercentualGrowth,
                    'Franchise': franchise,
                    'Type': adapt_type,
                    'AdaptTitle': adaptTitle,
                    'ReleaseDate': releaseDate,
                    'Rating': rating,
                    'NumVotes(k)': numVotes
                }])
                df_game_adaptation = pd.concat([df_game_adaptation, new_row], ignore_index=True)

# Remove linhas onde 'AdaptReleaseAbsoluteGrowth' e 'AdaptReleasePercentualGrowth' são NaN
df_game_adaptation = df_game_adaptation.dropna(subset=['AdaptReleaseAbsoluteGrowth', 'AdaptReleasePercentualGrowth'])
 
output_path = os.path.join(DIR_OUTPUT_TABLES, 'game_adaptation.csv')
df_game_adaptation.to_csv(
    output_path,
    index=False,
    encoding='utf-8'
)
df_game_adaptation.info()
print(f'DataFrame salvo em: {output_path}')

/tmp/ipykernel_246614/2715388290.py:216: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_game_adaptation = pd.concat([df_game_adaptation, new_row], ignore_index=True)
/tmp/ipykernel_246614/2715388290.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_game_adaptation = pd.concat([df_game_adaptation, new_row], ignore_index=True)
/tmp/ipykernel_246614/2715388290.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, thi

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, 0 to 104
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   GameTitle                              86 non-null     object 
 1   MaxHistoricalPeak                      86 non-null     object 
 2   AdaptReleasePeak                       86 non-null     object 
 3   AdaptReleaseAbsoluteGrowth             86 non-null     float64
 4   AdaptReleasePercentualGrowth           86 non-null     float64
 5   PreviousYearPeak                       82 non-null     object 
 6   PreviousYearAbsoluteGrowth             81 non-null     float64
 7   PreviousYearPercentualGrowth           81 non-null     float64
 8   NextMonthPeak                          85 non-null     object 
 9   NextMonthAbsoluteGrowth                85 non-null     float64
 10  NextMonthPercentualGrowth              85 non-null     float64
 11  NextMonthPre

/tmp/ipykernel_246614/2715388290.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_game_adaptation = pd.concat([df_game_adaptation, new_row], ignore_index=True)
/tmp/ipykernel_246614/2715388290.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_game_adaptation = pd.concat([df_game_adaptation, new_row], ignore_index=True)
/tmp/ipykernel_246614/2715388290.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, thi